In [ ]:
import pandas as pd

In [ ]:
df= pd.read_csv("train.csv")

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

# Get the list of categorical columns excluding the column "NObeyesdad"
categorical_columns = [col for col in df.select_dtypes(include=['object']).columns ] #if col != "NObeyesdad"

# Initialize OrdinalEncoder
ordinal_encoder = OrdinalEncoder()

# Fit and transform OrdinalEncoder on the categorical columns
df[categorical_columns] = ordinal_encoder.fit_transform(df[categorical_columns])


In [ ]:
df = df.drop(['id'], axis=1)

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import MinMaxScaler

# scale the data
scaler = MinMaxScaler()
scaledData = scaler.fit_transform(df)

In [ ]:
# fit the model
outliers_fraction = 0.05
clf = IsolationForest(contamination=outliers_fraction, random_state=42)
clf.fit(scaledData)

# predictions
y_pred = clf.predict(scaledData)

In [ ]:
y_pred_df = pd.DataFrame(y_pred, columns=['anomaly'])

In [ ]:
scaledData= pd.DataFrame(scaledData, columns=df.columns)

In [ ]:
scaledData['anomaly'] = y_pred

In [ ]:
scaledData

In [ ]:
scaledData['NObeyesdad'] = df['NObeyesdad']

In [ ]:
scaledData = scaledData[scaledData['anomaly'] != -1]

In [ ]:
label_mapping = {0: 'Insufficient_Weight'
                 , 1: 'Normal_Weight'
                 , 2: 'Obesity_Type_I' 
                 , 3: 'Obesity_Type_II'
                 , 4: 'Obesity_Type_III'
                 , 5: 'Overweight_Level_I'
                 , 6: 'Overweight_Level_II'}


# Transform encoded labels back to original labels using the mapping
scaledData["NObeyesdad"] = scaledData["NObeyesdad"].map(label_mapping)

In [ ]:
scaledData = scaledData.drop(['anomaly'], axis=1)


In [ ]:
lable = scaledData['NObeyesdad']

In [ ]:
scaledData = scaledData.drop(['NObeyesdad'], axis=1)

In [ ]:
lable = pd.DataFrame(lable)

In [ ]:
from sklearn.feature_selection import SelectPercentile, chi2
# now select features based on top 10 percentile
selector = SelectPercentile(chi2, percentile=100)
X_new = selector.fit_transform(scaledData, lable)


In [ ]:
# Get the indices of the selected features
selected_indices = selector.get_support(indices=True)

# Get the corresponding column names from scaledData
selected_column_names = scaledData.columns[selected_indices]

# Create a new DataFrame with selected features and corresponding column names
X_new = pd.DataFrame(X_new, columns=selected_column_names)
X_new

In [ ]:
X_new

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
# Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)


# Define the number of splits
n_splits = 5  # You can adjust this as needed

# Initialize the StratifiedShuffleSplit object
stratified_splitter = StratifiedShuffleSplit(n_splits=n_splits, test_size=0.2, random_state=42)
accuracy_scores = []
# Perform the split
for train_index, test_index in stratified_splitter.split(X_new, lable):
    X_train, X_test = X_new.iloc[train_index], X_new.iloc[test_index]
    y_train, y_test = lable.iloc[train_index], lable.iloc[test_index]

    # Fit the classifier on the training data
    rf_classifier.fit(X_train, y_train)
    # Predict the labels for the test data
    y_pred = rf_classifier.predict(X_test)
    
    # Calculate the accuracy of the model for this split
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)
# Calculate the mean accuracy across all splits
mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
print("Mean accuracy:", mean_accuracy)

In [154]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
# Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)


# Define the number of splits
n_splits = 5  # You can adjust this as needed

# Initialize the StratifiedShuffleSplit object
stratified_splitter = StratifiedShuffleSplit(n_splits=n_splits, test_size=0.2, random_state=42)
accuracy_scores = []
# Perform the split
for train_index, test_index in stratified_splitter.split(X_new, lable):
    X_train, X_test = X_new.iloc[train_index], X_new.iloc[test_index]
    y_train, y_test = lable.iloc[train_index], lable.iloc[test_index]

    # Fit the classifier on the training data
    rf_classifier.fit(X_train, y_train)
    # Predict the labels for the test data
    y_pred = rf_classifier.predict(X_test)
    
    # Calculate the accuracy of the model for this split
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)
# Calculate the mean accuracy across all splits
mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
print("Mean accuracy:", mean_accuracy)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_9640\2100561453.py:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_classifier.fit(X_train, y_train)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_9640\2100561453.py:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_classifier.fit(X_train, y_train)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_9640\2100561453.py:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_classifier.fit(X_train, y_train)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_9640\2100561453.py:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using rav

Mean accuracy: 0.9051724137931034


In [156]:
test = pd.read_csv('test.csv')

In [157]:
id = test["id"]
id = pd.DataFrame(id)
test = test.drop(["id"], axis=1)

In [158]:
test = test[X_new.columns]

In [159]:
# Get the list of categorical columns excluding the column "NObeyesdad"
categorical_columns = [col for col in test.select_dtypes(include=['object']).columns ] #if col != "NObeyesdad"

# Initialize OrdinalEncoder
ordinal_encoder = OrdinalEncoder()

# Fit and transform OrdinalEncoder on the categorical columns
test[categorical_columns] = ordinal_encoder.fit_transform(test[categorical_columns])


In [163]:
prediction = rf_classifier.predict(test)

prediction = pd.DataFrame(prediction, columns=["NObeyesdad"])


In [164]:
prediction

,NObeyesdad
0,3.0
1,4.0
2,4.0
3,3.0
4,4.0
...,...
13835,3.0
13836,3.0
13837,4.0
13838,2.0


In [165]:
label_mapping = {0: 'Insufficient_Weight'
                 , 1: 'Normal_Weight'
                 , 2: 'Obesity_Type_I' 
                 , 3: 'Obesity_Type_II'
                 , 4: 'Obesity_Type_III'
                 , 5: 'Overweight_Level_I'
                 , 6: 'Overweight_Level_II'}


# Transform encoded labels back to original labels using the mapping
prediction["NObeyesdad"] = prediction["NObeyesdad"].map(label_mapping)

In [171]:
submition = pd.concat([id, prediction], axis=1)
submition

,id,NObeyesdad
0,20758,Obesity_Type_II
1,20759,Obesity_Type_III
2,20760,Obesity_Type_III
3,20761,Obesity_Type_II
4,20762,Obesity_Type_III
...,...,...
13835,34593,Obesity_Type_II
13836,34594,Obesity_Type_II
13837,34595,Obesity_Type_III
13838,34596,Obesity_Type_I


In [173]:
submition.to_csv('Out2.csv', index=False)